## YT Video to TXT file conversion

In [ ]:
# Install Required Dependencies
# -----------------------------
# Ensure you have Python 3.7+ installed before running this script.
# The following command installs all necessary libraries:
# - yt-dlp: A video downloader and subtitle extractor for YouTube and other sites.
# - pyphen: A library for syllable segmentation, useful for karaoke files.
# - numpy: A numerical library for handling arrays and generating random values.

!pip install --upgrade pip  # Ensure pip is up to date
!pip install yt-dlp pyphen numpy  

# Import Required Libraries
# -------------------------
import os       # Provides functions for interacting with the file system
import numpy as np  # Used for numerical operations (e.g., generating random values)
import yt_dlp   # Enables video downloading and subtitle extraction
import pyphen   # Allows syllable-based text processing (for karaoke synchronization)

# Additional Requirements (Manual Installation)
# ---------------------------------------------
# Some functionalities may require external tools like ffmpeg:
# - ffmpeg is required for processing audio and video files.
# - Install it manually if needed:
#   - Windows: Download from https://ffmpeg.org/download.html
#   - macOS: Install using `brew install ffmpeg` (requires Homebrew: https://brew.sh/)
#   - Linux: Install using `sudo apt install ffmpeg` (Debian/Ubuntu) or `sudo dnf install ffmpeg` (Fedora)

# Uncomment the following line to check if ffmpeg is installed
# !ffmpeg -version  

In [ ]:

def list_available_subtitles(youtube_url):
    try:
        ydl_opts = {'list_subs': True, 'quiet': True}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(youtube_url, download=False)
            subtitles = info_dict.get('subtitles', {})
            auto_subtitles = info_dict.get('automatic_captions', {})
            
            all_subs = {**subtitles, **auto_subtitles}
            if not all_subs:
                print("No subtitles available for this video.")
                return []
            
            available_langs = list(all_subs.keys())
            print("Available subtitle languages:")
            for lang in available_langs:
                print(f"- {lang}")
            return available_langs
    except Exception as e:
        print(f"Error listing subtitles: {e}")
        return []

def download_youtube_subtitles(youtube_url, lang, output_directory='./', ffmpeg_location=None):
    ydl_opts = {
        'writeautomaticsub': True,
        'writesubtitles': True,
        'subtitleslangs': [lang],
        'outtmpl': 'song_temp.%(ext)s',
        'quiet': False,
        'ffmpeg_location': ffmpeg_location,
    }

    os.makedirs(output_directory, exist_ok=True)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([youtube_url])
        except Exception as e:
            raise Exception(f"Error downloading subtitles: {str(e)}")
    
    subtitle_file = f"song_temp.{lang}.vtt"
    subtitle_file_path = os.path.join(output_directory, subtitle_file)
    
    if os.path.exists(subtitle_file_path):
        return subtitle_file_path
    else:
        raise FileNotFoundError(f"Subtitles not found for {youtube_url}.")

def parse_vtt(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    lyrics = []
    timestamps = []
    timestamp = None
    for line in lines:
        if "-->" in line:
            timestamp = line.strip()
            timestamps.append(timestamp.split(" --> ")[0])
        elif line.strip():
            if timestamp:
                lyrics.append((timestamp, line.strip()))
                timestamp = None
    
    return lyrics, timestamps

def time_to_beats(time_str, bpm):
    h, m, s = map(float, time_str.replace(',', '.').split(":"))
    total_seconds = h * 3600 + m * 60 + s
    beats = total_seconds * (bpm / 60)
    return int(beats)

def time_to_ms(time_str):
    h, m, s = map(float, time_str.replace(',', '.').split(":"))
    total_ms = int((h * 3600 + m * 60 + s) * 1000)
    return total_ms

def estimate_bpm(timestamps):
    if len(timestamps) < 2:
        return 120  # Valor per defecte si no hi ha prou timestamps
    
    time_diffs = [time_to_ms(timestamps[i]) - time_to_ms(timestamps[i - 1]) for i in range(1, len(timestamps))]
    avg_time_diff = sum(time_diffs) / len(time_diffs) if time_diffs else 500
    
    beats_per_second = 1000 / avg_time_diff
    bpm = beats_per_second * 60
    return int(bpm)

def syllabify(text):
    dic = pyphen.Pyphen(lang='en')
    return dic.inserted(text).split("-")

def extract_ultrastar_format(title, artist, youtube_url, lyrics, bpm, language='English', genre='Pop', year='2024', creator='AutoGen', cover_image='', first_lyric_start_ms=0):
    output = [
        f"#ARTIST:{artist}",
        f"#TITLE:{title}",
        f"#LANGUAGE:{language}",
        f"#GENRE:{genre}",
        f"#YEAR:{year}",
        f"#CREATOR:{creator}",
        f"#MP3:{artist} - {title}.mp3",
        f"#COVER:{cover_image}",
        f"#BPM:{bpm}",
        f"#GAP:{first_lyric_start_ms}"
    ]
    
    prev_beat = 0
    for timestamp, text in lyrics:
        start_time, end_time = timestamp.split(" --> ")
        syllables = syllabify(text)
        start_beat = time_to_beats(start_time, bpm)
        end_beat = time_to_beats(end_time, bpm)
        duration = max(end_beat - start_beat, 1)
        
        for i, syllable in enumerate(syllables):
            tone = np.random.randint(5, 20)
            output.append(f": {start_beat + i} {duration} {tone} {syllable}")
        
        if (start_beat - prev_beat) > 30:
            output.append("- 0")
            prev_beat = start_beat
    
    output.append("E")
    return "\n".join(output)

def main():
    youtube_url = input("Enter the YouTube video URL: ").strip()
    available_langs = list_available_subtitles(youtube_url)
    
    if not available_langs:
        print("Exiting: No subtitles available.")
        return
    
    print("Select a subtitle language from the list above:")
    lang = input("Enter the language code (e.g., 'en', 'es'): ").strip()
    if lang not in available_langs:
        print("Invalid selection. Exiting.")
        return
    
    title = input("Enter the song title: ").strip()
    artist = input("Enter the artist name: ").strip()
    genre = input("Enter the genre: ").strip()
    year = input("Enter the release year: ").strip()
    creator = input("Enter the creator name: ").strip()
    cover_image = input("Enter the cover image filename: ").strip()
    output_directory = input("Enter the output directory (default is './'): ").strip() or './'
    ffmpeg_location = input("Enter the full path to ffmpeg (leave blank if not required): ").strip() or None
    
    safe_title = title.replace(" ", "_").lower()
    safe_artist = artist.replace(" ", "_").lower()
    output_filename = f"{safe_artist}_{safe_title}_ultrastar.txt"
    output_filepath = os.path.join(output_directory, output_filename)
    
    try:
        subtitle_file = download_youtube_subtitles(youtube_url, lang, output_directory=output_directory, ffmpeg_location=ffmpeg_location)
        lyrics, timestamps = parse_vtt(subtitle_file)
        
        if not lyrics:
            raise Exception("No lyrics found in the subtitle file.")
        
        first_lyric_start_ms = time_to_ms(timestamps[0]) if timestamps else 0
        bpm = estimate_bpm(timestamps)
        
        ultrastar_txt = extract_ultrastar_format(title, artist, youtube_url, lyrics, bpm, lang, genre, year, creator, cover_image, first_lyric_start_ms)
        
        with open(output_filepath, "w", encoding="utf-8") as f:
            f.write(ultrastar_txt)
        
        print(f"UltraStar file saved to: {output_filepath}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


In [ ]:
 ## Example Params
    
    # Provide the YouTube video URL.
    #video_url = "https://www.youtube.com/watch?v=F3aXpa1rQEY"
    
    # Provide the song title and artist name.
    #video_title = "the_man"
    #artist_name = "taylor_swift"
    
    # Set your desired output directory.
    # output_dir = r"C:\Users\usuario\Desktop\"
    
    # Provide the full path to your FFmpeg executable (required for merging streams).
    # ffmpeg_path = r'C:\Users\usuario\Anaconda3\envs\songtotxt\Library\bin\ffmpeg.exe'
